In [4]:
import findspark
findspark.init()

In [5]:
from pyspark import SparkContext
sc = SparkContext("local", "SparkRddTuple")

22/05/13 22:02:35 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/13 22:02:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/13 22:02:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/13 22:02:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
rdd = sc.textFile("hdfs://localhost:9000/ml-latest-small/movies.csv")


In [7]:
result = rdd.count()
print(result)

9743


In [8]:
# Answer to #1.1

#print(rdd.glom().collect())
print(rdd.take(5))

['movieId,title,genres', '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


In [9]:
print(rdd.count())

9743


In [10]:
print(rdd.take(5))

['movieId,title,genres', '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


In [11]:
headerLine = rdd.first()
rddList = rdd.filter (lambda line: line != headerLine)
print(rddList.take(5))


['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance', '5,Father of the Bride Part II (1995),Comedy']


22/05/13 22:02:57 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:531)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3441)
	at org.apache.hadoop.hdfs.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:777)
	at org.apache.hadoop.hdfs.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:694)
	at org.apache.hadoop.hdfs.BlockReaderFactory.build(BlockReaderFactory.java:355)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:665)
	at org.apache.hadoop.hdfs.DFSInputStream.seekToBlockSource(DFSInputStream.java:1567)
	at org.apache.hadoop.hdfs.DFSInputStream.re

In [12]:
wordListRdd = rddList.map (lambda line: line.split(","))

wordListRdd.take(5)


[['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy']]

In [13]:
wordRdd = wordListRdd.flatMap(lambda elements: elements)
filteredRDD = wordRdd.filter(lambda x: x.startswith('Adventure'))

filteredRDD.count() # Answer to 1.2


658

In [14]:
romanceFilterRDD = wordRdd.filter(lambda x: "Romance" in x)
romanceFilterRDD.count() # Answer to 1.3

1603

In [16]:
comedyFilterRDD = wordRdd.filter(lambda x: "Comedy" in x)
comedyFilterRDD.count() # Answer to 1.2

3768

In [17]:
ratingRdd = sc.textFile("hdfs://localhost:9000/ml-latest-small/ratings.csv")


In [18]:
print(ratingRdd.take(10)) # Answer to 1.4

['userId,movieId,rating,timestamp', '1,1,4.0,964982703', '1,3,4.0,964981247', '1,6,4.0,964982224', '1,47,5.0,964983815', '1,50,5.0,964982931', '1,70,3.0,964982400', '1,101,5.0,964980868', '1,110,4.0,964982176', '1,151,5.0,964984041']


In [19]:
ratingListRdd = ratingRdd.map (lambda line: line.split(","))
print(ratingListRdd.take(5))


[['userId', 'movieId', 'rating', 'timestamp'], ['1', '1', '4.0', '964982703'], ['1', '3', '4.0', '964981247'], ['1', '6', '4.0', '964982224'], ['1', '47', '5.0', '964983815']]


In [20]:
ratingFilterRDD = ratingListRdd.filter(lambda x: x[2] >= '1.0')

print(ratingFilterRDD.take(20)) # Answer to 1.4

[['userId', 'movieId', 'rating', 'timestamp'], ['1', '1', '4.0', '964982703'], ['1', '3', '4.0', '964981247'], ['1', '6', '4.0', '964982224'], ['1', '47', '5.0', '964983815'], ['1', '50', '5.0', '964982931'], ['1', '70', '3.0', '964982400'], ['1', '101', '5.0', '964980868'], ['1', '110', '4.0', '964982176'], ['1', '151', '5.0', '964984041'], ['1', '157', '5.0', '964984100'], ['1', '163', '5.0', '964983650'], ['1', '216', '5.0', '964981208'], ['1', '223', '3.0', '964980985'], ['1', '231', '5.0', '964981179'], ['1', '235', '4.0', '964980908'], ['1', '260', '5.0', '964981680'], ['1', '296', '3.0', '964982967'], ['1', '316', '3.0', '964982310'], ['1', '333', '5.0', '964981179']]


In [21]:
ratingRdd2 = ratingFilterRDD.map (lambda line: tuple(line))
print(ratingRdd2.take(5))


[('userId', 'movieId', 'rating', 'timestamp'), ('1', '1', '4.0', '964982703'), ('1', '3', '4.0', '964981247'), ('1', '6', '4.0', '964982224'), ('1', '47', '5.0', '964983815')]


In [22]:
headerLine = ratingRdd2.first()
rddContent = ratingRdd2.filter (lambda line: line != headerLine)

ratingRdd3 = rddContent.map(lambda n: ((n[1], float(n[2]))))
print(ratingRdd3.take(10))
                                
#result = ratingRdd3.countByKey() 
#print(result.take(5))


[('1', 4.0), ('3', 4.0), ('6', 4.0), ('47', 5.0), ('50', 5.0), ('70', 3.0), ('101', 5.0), ('110', 4.0), ('151', 5.0), ('157', 5.0)]


In [26]:
result2 = ratingRdd3.reduceByKey(lambda acc, value: acc + value)
result2.take(5)  # Answer to 1.5

[('1', 842.5), ('3', 169.0), ('6', 402.5), ('47', 806.0), ('50', 864.5)]

In [27]:
def add(acc, value):
    output = acc + value
    print("acc", acc, "value", value, "output", output)
    return output


In [34]:
result2.take(5)




[('1', 842.5), ('3', 169.0), ('6', 402.5), ('47', 806.0), ('50', 864.5)]

In [37]:
from operator import add



In [38]:
result2.foldByKey(0, add).collect() # Answer to 1.6

[('1', 842.5),
 ('3', 169.0),
 ('6', 402.5),
 ('47', 806.0),
 ('50', 864.5),
 ('70', 193.0),
 ('101', 87.0),
 ('110', 955.0),
 ('151', 156.0),
 ('157', 31.5),
 ('163', 235.0),
 ('216', 163.0),
 ('223', 401.0),
 ('231', 406.5),
 ('235', 257.5),
 ('260', 1061.5),
 ('296', 1287.5),
 ('316', 472.5),
 ('333', 189.0),
 ('349', 396.5),
 ('356', 1369.5),
 ('362', 120.0),
 ('367', 499.5),
 ('423', 28.5),
 ('441', 165.0),
 ('457', 758.0),
 ('480', 892.0),
 ('500', 487.5),
 ('527', 928.5),
 ('543', 135.5),
 ('552', 199.0),
 ('553', 248.0),
 ('590', 629.0),
 ('592', 648.0),
 ('593', 1159.5),
 ('596', 206.0),
 ('608', 745.0),
 ('648', 572.5),
 ('661', 169.0),
 ('673', 142.0),
 ('733', 440.0),
 ('736', 408.5),
 ('780', 695.0),
 ('804', 26.0),
 ('919', 357.0),
 ('923', 279.0),
 ('940', 32.0),
 ('943', 27.5),
 ('954', 73.5),
 ('1009', 27.5),
 ('1023', 47.0),
 ('1024', 22.0),
 ('1025', 89.5),
 ('1029', 118.5),
 ('1030', 41.5),
 ('1031', 83.5),
 ('1032', 135.0),
 ('1042', 137.5),
 ('1049', 73.0),
 ('106

In [210]:
rddContent.collect()
import datetime
timestamp = rddContent.map(lambda x: (x[0], x[1], x[2], datetime.datetime.fromtimestamp(int(x[3]))))
timestamp.take(5) # Answer to 1.7



[('1', '1', '4.0', datetime.datetime(2000, 7, 31, 0, 15, 3)),
 ('1', '3', '4.0', datetime.datetime(2000, 7, 30, 23, 50, 47)),
 ('1', '6', '4.0', datetime.datetime(2000, 7, 31, 0, 7, 4)),
 ('1', '47', '5.0', datetime.datetime(2000, 7, 31, 0, 33, 35)),
 ('1', '50', '5.0', datetime.datetime(2000, 7, 31, 0, 18, 51))]

In [ ]:
joinRdd = rdd.join(ratingRdd)
joinRdd.take(5)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 42808)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/spark-3.1.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/opt/spark-3.1.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/opt/spark-3.1.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 239

In [211]:
result2.take(5)  # Answer to 1.5

[('1', 842.5), ('3', 169.0), ('6', 402.5), ('47', 806.0), ('50', 864.5)]

In [224]:
print(result2.sortBy(lambda i:i[1],False).take(100)) # Answer to 1.9


[('318', 1404.0), ('356', 1369.5), ('296', 1287.5), ('2571', 1163.0), ('593', 1159.5), ('260', 1061.5), ('110', 955.0), ('2959', 930.0), ('527', 928.5), ('480', 892.0), ('1196', 889.0), ('589', 888.0), ('50', 864.5), ('1', 842.5), ('1198', 841.0), ('2858', 827.0), ('858', 823.5), ('4993', 812.0), ('1210', 811.0), ('47', 806.0), ('2028', 779.0), ('150', 773.0), ('7153', 762.0), ('457', 758.0), ('5952', 755.5), ('608', 745.0), ('32', 705.0), ('2762', 695.5), ('780', 695.0), ('588', 693.5), ('1270', 690.5), ('364', 678.0), ('3578', 668.5), ('4306', 657.0), ('4226', 655.0), ('592', 648.0), ('58559', 631.5), ('590', 629.0), ('380', 622.0), ('377', 603.0), ('1197', 601.0), ('79132', 581.0), ('1214', 579.0), ('1704', 575.0), ('1580', 574.5), ('648', 572.5), ('1291', 566.0), ('1136', 565.5), ('1265', 564.0), ('6539', 562.5), ('1036', 559.5), ('1193', 559.0), ('6377', 558.0), ('1089', 550.5), ('595', 550.5), ('1221', 549.5), ('7361', 544.5), ('2329', 544.0), ('1213', 535.5), ('293', 534.5), ('6

In [228]:
print(result2.filter(lambda i:i[1]<100).take(5)) # Answer to 1.10


[('101', 87.0), ('157', 31.5), ('423', 28.5), ('804', 26.0), ('940', 32.0)]
